### Init CosmWasm Client

In [ ]:
import { WebsocketClient, Tendermint37Client } from '@cosmjs/tendermint-rpc';

wsClient = await Tendermint37Client.create(new WebsocketClient('wss://rpc.orai.io'));

contracts = {
  engineAddr: 'orai1wrkchuss9wtph4mxrzqksfrulj7hsl89z0048hg8l7hcglse5rxqea2qnr',
  insuranceFundAddr: 'orai1l2z27tt0aq2vd2jr0g7vhy8975t6u3sly8pqay9ek3dctgpmkyrqju3dek',
  feePoolAddr: 'orai10q37uaq728y93u03dw6jzcxqqc36cu4q08k0c4wmhj4egqch69zstja6xu',
  pricefeedAddr: 'orai1s57duq6h0r0q6spfdhujnn695a3e9ka59zvv0yrvx7d80gvaf4hsfkezyr',
  oraiusdcVamm: 'orai1hgc4tmvuj6zuagyjpjjdrgwzj6ncgclm0n6rn4vwjg3wdxxyq0fs9k3ps9',
  injusdcVamm: 'orai1z36626k3s5k6nl0usn8543v67edn0rpgxnpr58xvr0luvdxu55cs96dv73',
  usdcAddr: 'orai15un8msx3n5zf9ahlxmfeqd2kwa5wm0nrpxer304m9nd5q6qq0g6sku5pdd'
};

### Extract transfer transaction from events

In [ ]:
import { buildQuery } from '@cosmjs/tendermint-rpc/build/tendermint37/requests.js';

getTransfer = async (query, page = undefined, limit = 10) => {
    const result = await wsClient.txSearch({
      query: buildQuery(query),
      page,
      per_page: limit
    });
    const wasmList = result.txs.flatMap((tx) => tx.result.events).filter((e) => e.type == 'wasm');    
    const map = Object.fromEntries(Object.entries(contracts).map(([k, v]) => [v, k]));
    const trans = [];
    for (const event of wasmList) {
    let set = false;
    for (const attr of event.attributes) {
        const key = atob(attr.key, 'base64');
        const value = atob(attr.value, 'base64');

        switch (key) {
        case 'action':
            if (value === 'transfer') {
            trans.push({});
            set = true;
            }
            break;
        case 'amount':
        case 'from':
        case 'to':
            if (!set) break;
            let displayValue = value;
            if (key == 'from' || key == 'to') {
            displayValue = map[displayValue] || displayValue;
            } else {
            displayValue = +displayValue;
            }
            trans[trans.length - 1][key] = displayValue;
            break;
        }
    }
    }

    return trans;
};

List Transfer Transactions

In [ ]:
const result = await getTransfer({
  tags: [
    {
      key: 'wasm._contract_address',
      value: contracts.engineAddr
    },
    {
      key: 'wasm.action',
      value: 'close_position'
    },
    {
      key: 'wasm.vamm',
      value: contracts.oraiusdcVamm
    }
  ]
});
console.table(result);